## 1. 랭체인 프롬프트 작성(MessagesPlaceholder)

In [9]:
import os
from dotenv import load_dotenv
load_dotenv()


from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash"
)
response = llm.invoke("1255, Who are you")
print(response.content)

I am a large language model, trained by Google.


In [7]:
import base64
img_path = r"C:\Users\Sese\Pictures\Screenshots\스크린샷 2025-06-07 195837.png"
img = open(img_path, "rb").read()
image_data = base64.b64encode(img).decode('utf-8')

prompt = ChatPromptTemplate([
    {
        "role":"user",
        "content":[
            {
                "type":"text",
                "text":"Who are you?"
            },
            {
                "type":"image",
                "source_type":"base64",
                "mime_type":"image/png",
                "data":"{image_data}"
            }
        ]
    }
])

chain = prompt | llm
response = chain.invoke({"image_data": image_data})
print(response.content)

I am an AI that can understand and respond to your queries. I can also process text from images using OCR. In this case, I have recognized some Python code defining a class called `DebateBot` that uses the `langchain` library to interact with the `ChatOpenAI` model. The code also shows a debate between two bots, one pro-Apple and one pro-Samsung, using strong, potentially offensive language.


## 2. Tool 호출

In [10]:
from langchain_experimental.tools import PythonREPLTool
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda
from langchain_openai import ChatOpenAI


llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash"
)

python_tool = PythonREPLTool()
print(python_tool.invoke("print(24/24)"))

Python REPL can execute arbitrary code. Use with caution.


1.0



In [26]:
def print_and_execute(code, debug=True):
    if debug:
        print("CODE:")
        print(code)
    return python_tool.invoke(code)

def print_and_execute(code, debug=True):
    if debug:
        print("CODE:")
        print(code)
    return python_tool.invoke(code)

prompt = ChatPromptTemplate([
    {
        "role":"user",
        "content": [
            {
                "type":"text",
                "text":"""너는 파이썬 코드를 작성하는 프로그레머야.
REPL 환경에서 작동하는 파이썬 코드를 작성해줘.
이 떄 코드는 clean하고 실행가능해야 해.
'if __name__ == "__main__":' 또는 다른 file-level guard는 절대 사용하지 마
code를 다른 설명이나 markdown, formatting 없이 직접적으로 작성해줘."""
            },
            {
                "type":"text",
                "text":"{input}"
            }
        ]
    }
])
chain = prompt | llm | StrOutputParser() | RunnableLambda(print_and_execute)
print(chain.invoke({"input":"로또 번호 생성기를 출력하는 코드를 작성하세요."}))

CODE:
```python
import random

def generate_lotto_numbers():
    """Generates a set of 6 unique random numbers between 1 and 45 (inclusive)."""
    return sorted(random.sample(range(1, 46), 6))

# Generate and print the lotto numbers
lotto_numbers = generate_lotto_numbers()
print(lotto_numbers)
```
[5, 14, 20, 21, 27, 40]



In [35]:
from langchain_community.utilities.dalle_image_generator import DallEAPIWrapper
from IPython.display import Image

dalle = DallEAPIWrapper(
    model="dall-e-3",
    size='1024x1024',
    quality='standard',
    n=1
)

llm = ChatGoogleGenerativeAI(
    model = 'gemini-2.0-flash'
)

prompt = ChatPromptTemplate([
    ("user",    "Generate a detailed IMAGE GENERATION prompt for DALL-E based on the following description. "),
    ("user", "Return only the prompt, no intro, no explanation, no chatty, no markdown, no code block, no nothing. Just the prompt"),
    ("user","Output should be less than 1000 characters. Write in English only."),
    ("user", "Image Description: \n{image_desc}")
])
chain = prompt | llm | StrOutputParser() | RunnableLambda(lambda prompt: dalle.run(prompt))
image_url = chain.invoke({"image_desc": \
    "A futuristic cityscape with flying cars and neon lights, at night."})
Image(url=image_url, width=400)


## 3. 사용자 정의 도구 (Custom Tool)

In [49]:
from langchain.agents import tool

@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)

@tool
def add(a:float, b:float)-> float:
    """Adds two numbers together"""
    return a+b

import re
import requests
from bs4 import BeautifulSoup
@tool
def naver_news_crawl(news_url: str) -> str:
    """Crawls a 네이버 (naver.com) news article and returns the body content."""
    # HTTP GET 요청 보내기
    response = requests.get(news_url)

    # 요청이 성공했는지 확인
    if response.status_code == 200:
        # BeautifulSoup을 사용하여 HTML 파싱
        soup = BeautifulSoup(response.text, "html.parser")

        # 원하는 정보 추출
        title = soup.find("h2", id="title_area").get_text()
        content = soup.find("div", id="contents").get_text()
        cleaned_title = re.sub(r"\n{2,}", "\n", title)
        cleaned_content = re.sub(r"\n{2,}", "\n", content)
    else:
        print(f"HTTP 요청 실패. 응답 코드: {response.status_code}")

    return f"{cleaned_title}\n{cleaned_content}"


tools = [get_word_length, add, naver_news_crawl]

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")
llm_with_tools = llm.bind_tools(tools)

response = llm_with_tools.invoke("What is the length of the word 'hello'?")
display(response.tool_calls)
display(response.__dict__)

[{'name': 'get_word_length',
  'args': {'word': 'hello'},
  'id': 'ba4f6327-5dcd-4ef7-af1e-7e0198794f1e',
  'type': 'tool_call'}]

{'content': '',
 'additional_kwargs': {'function_call': {'name': 'get_word_length',
   'arguments': '{"word": "hello"}'}},
 'response_metadata': {'prompt_feedback': {'block_reason': 0,
   'safety_ratings': []},
  'finish_reason': 'STOP',
  'model_name': 'gemini-2.0-flash',
  'safety_ratings': []},
 'type': 'ai',
 'name': None,
 'id': 'run--433c8583-2814-4882-9c56-0ed507854dfe-0',
 'example': False,
 'tool_calls': [{'name': 'get_word_length',
   'args': {'word': 'hello'},
   'id': 'ba4f6327-5dcd-4ef7-af1e-7e0198794f1e',
   'type': 'tool_call'}],
 'invalid_tool_calls': [],
 'usage_metadata': {'input_tokens': 69,
  'output_tokens': 7,
  'total_tokens': 76,
  'input_token_details': {'cache_read': 0}}}

In [57]:
def execute_tool_calls(tool_call_results):
    display(tool_call_results)
    """
    도구 호출 결과를 실행하는 함수
    
    :param_tool_call_results: 도구 호출 결과 리스트
    :param_tools: 사용가능한 도구 리스트
    """
    for tool_call_result in tool_call_results:
        tool_name = tool_call_result['type']
        tool_args = tool_call_result['args']
        
        matching_tool = next((tool for tool in tools if tool.name == tool_name), None)
        if matching_tool:
            result = matching_tool.invoke(tool_args)
            print(f"Tool: {tool_name}, Result: {result}")
        else:
            print(f"No matching tool found for: {tool_name}")

from langchain_core.output_parsers.openai_tools import JsonOutputToolsParser
chain = llm_with_tools | JsonOutputToolsParser() | execute_tool_calls

chain.invoke("What is the length of the word 'teddynote'?")
chain.invoke("114.5 + 121.2")
chain.invoke(
    "뉴스 기사 내용을 크롤링해줘: https://n.news.naver.com/mnews/hotissue/article/092/0002347672?type=series&cid=2000065"
)

[{'args': {'word': 'teddynote'}, 'type': 'get_word_length'}]

Tool: get_word_length, Result: 9


[{'args': {'b': 121.2, 'a': 114.5}, 'type': 'add'}]

Tool: add, Result: 235.7


[{'args': {'news_url': 'https://n.news.naver.com/mnews/hotissue/article/092/0002347672?type=series&cid=2000065'},
  'type': 'naver_news_crawl'}]

Tool: naver_news_crawl, Result: [미장브리핑] 9월 미국 CPI 주목…3분기 S&P500 실적 발표

			▲10일(현지시간) 미국 9월 소비자물가지수(CPI) 발표 예정. 고용 지표가 양호하게 나온 가운데 물가 지표 주목. 9월 미국 비농업고용 25만4천명 증가해 시장 예상치 14만명 크게 상회. 이는 6개월 래 최대 규모로 지난 12개월 평균값 20만3천명 증가한 것보다도 높은 수치. 9월 실업률은 4.1%로 2개월 연속 하락했으며, 평균 시간당 임금은 전년 동월 대비 4% 증가해 5월 이후 최고 수준.▲시장에서 9월 헤드라인 CPI는 8월 전년 동월 대비 2.6% 로 5개월 연속 둔화하고 9월에는 2.3% 증가로 추가 하락 예상. 전월 대비도 8월 0.2% 둔화 예상. 근원 CPI는 지난 8월 3.2%와 비슷한 수준 관측.▲11일에는 미국 9월 제조업물가지수(PPI) 발표. 지난 6월 부터 8월까지 반등 추세 꺾여. 8월은 1.7% 증가.
(사진=이미지투데이)▲11월 미국 연방준비제도(연준) 공개시장위원회(FOMC) 에서 0.50%p 인하 기대가 크케 후퇴한 가운데, 9일에는 FOMC 의사록 공개. 지난 9월 회의에서 빅컷(0.50%p) 단행한 배경과 인플레이션 전망에 대한 논의를 알 수 있을 것으로 보여.▲미국 스탠다드앤푸어스(S&P) 500 기업의 3분기 실적 발표 시작. 평균 이익증가율 추정치는 전년 동기 대비 4.6%로 5개분기 연속 플러스이나 증가폭은 둔화 예상. 11일부터 JP모건체이스, 웰스파고 등 대형은행들의 실적 발표.▲FTSE 러셀은 8일 정례 시장분류 결과를 발표. 한국은 2022년 관찰대상국 지정 이후 금번 시장접근성 등급(L1) 상향으로 세계국채지수(WGBI) 에 편입될 지 관심. 주식의 경우 지난 2009년부터 선진국 지수에 편입돼 있는 한국 증시에 대해 공매도 제한 등을 이유로 관찰 대상국으로 지정할지 관심. 지정되더라도 검토 기간이 있어 즉각 제외되지는 않음.
손희연 기자(kunst@zdnet.co.k

In [56]:
import json
from langchain.output_parsers.openai_tools import JsonOutputToolsParser

# JSON 문자열
json_string = '''
{
  "tool_calls": [
    {
      "id": "call_abc",
      "type": "function",
      "function": {
        "name": "search_weather",
        "arguments": "{\\"location\\": \\"Seoul\\"}"
      }
    }
  ]
}
'''

# 문자열 -> Python dict
parsed_json = json.loads(json_string)

print(parsed_json)


{'tool_calls': [{'id': 'call_abc', 'type': 'function', 'function': {'name': 'search_weather', 'arguments': '{"location": "Seoul"}'}}]}


## 4. Agent & AgentExector

In [58]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but don't know current events",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)


In [59]:
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor

tools = [get_word_length, add, naver_news_crawl]

agent = create_tool_calling_agent(llm, tools, prompt)

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
)


In [ ]:
result = agent_executor.invoke({"input": "뉴스 기사 내용을 크롤링해줘: https://n.news.naver.com/mnews/hotissue/article/092/0002347672?type=series&cid=2000065\n 한 다음에 글자수 세줘"})
print(result["output"])




> Entering new AgentExecutor chain...

Invoking: `naver_news_crawl` with `{'news_url': 'https://n.news.naver.com/mnews/hotissue/article/092/0002347672?type=series&cid=2000065'}`


[미장브리핑] 9월 미국 CPI 주목…3분기 S&P500 실적 발표

			▲10일(현지시간) 미국 9월 소비자물가지수(CPI) 발표 예정. 고용 지표가 양호하게 나온 가운데 물가 지표 주목. 9월 미국 비농업고용 25만4천명 증가해 시장 예상치 14만명 크게 상회. 이는 6개월 래 최대 규모로 지난 12개월 평균값 20만3천명 증가한 것보다도 높은 수치. 9월 실업률은 4.1%로 2개월 연속 하락했으며, 평균 시간당 임금은 전년 동월 대비 4% 증가해 5월 이후 최고 수준.▲시장에서 9월 헤드라인 CPI는 8월 전년 동월 대비 2.6% 로 5개월 연속 둔화하고 9월에는 2.3% 증가로 추가 하락 예상. 전월 대비도 8월 0.2% 둔화 예상. 근원 CPI는 지난 8월 3.2%와 비슷한 수준 관측.▲11일에는 미국 9월 제조업물가지수(PPI) 발표. 지난 6월 부터 8월까지 반등 추세 꺾여. 8월은 1.7% 증가.
(사진=이미지투데이)▲11월 미국 연방준비제도(연준) 공개시장위원회(FOMC) 에서 0.50%p 인하 기대가 크케 후퇴한 가운데, 9일에는 FOMC 의사록 공개. 지난 9월 회의에서 빅컷(0.50%p) 단행한 배경과 인플레이션 전망에 대한 논의를 알 수 있을 것으로 보여.▲미국 스탠다드앤푸어스(S&P) 500 기업의 3분기 실적 발표 시작. 평균 이익증가율 추정치는 전년 동기 대비 4.6%로 5개분기 연속 플러스이나 증가폭은 둔화 예상. 11일부터 JP모건체이스, 웰스파고 등 대형은행들의 실적 발표.▲FTSE 러셀은 8일 정례 시장분류 결과를 발표. 한국은 2022년 관찰대상국 지정 이후 금번 시장접근성 등급(L1) 상향으로 세계국